# [Simple Models]

*KU LeeDongGyu*

### Contents

1. MLP
```
1) Size = 64
2) Size = 48.
```
2. Simple CNN
```
1) Size = 64
2) Size = 48
```
3. Size = 48 , No Early Stopping
```
1) Epoch = 50
      (1) MLP
      (2) Simple CNN
2) Epoch = 100
      (1) MLP
      (2) Simple CNN
```


### Module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [4]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [0]:
from f1score import macro_f1score, weighted_f1score

## 1. MLP
---

In [0]:
###################### 1. model - MLP ######################

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48]) / 255
x_valid = np.array(x_valid).reshape([-1,48,48]) / 255
x_test = np.array(x_test).reshape([-1,48,48]) / 255

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
size=64
x_train = np.array(x_train).reshape([-1,48,48,1])

In [10]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [12]:
x_train_zoom.shape

(28698, 64, 64, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [14]:
x_valid_zoom.shape

(3589, 64, 64, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

#### 1) Size = 64

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(64,64,1))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4096)              0         
_________________________________________________________________
d1 (Dense)                   (None, 128)               524416    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
d2 (Dense)                   (None, 512)               66048     
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
d3 (Dense)                   (None, 7)                 3591

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')


# early stopping의 옵션은 다음과 같다.
# monitor : 관찰하고자 하는 항목. 'val_loss', 'val_acc' 가 기본이고, 내가 정의하는 옵션을 기준으로 삼을수도 있다.
# mode라는 옵션을 통해, 최소를 정할지 최대를 정할지 구체화해서 정할 수 있으며, mode옵션을 주지 않으면 auto가 default이므로, 알아서 정해줌
# 우리는 mode='max'임.

# min_delta : 개선되고 있다고 판단하기 위한 최소 변화량을 나타낸다.
# 만약 변화량이 min_delta보다 적은 경우에는 개선이 없다고 판단한다.

# patience : 개선이 없다고 바로 종료하지 않고 개선이 없는 epoch를 얼마나 기다려 줄 것인 가를 지정한다.
# 만약 10이라고 지정하면 개선이 없는 epoch가 10회가 되도록 지속될 경우 학습일 종료한다.
# 데이터의 형태에 따라 잡는 방법이 다르지만, 보통 1~10의 숫자를 택한다.


# verbose : 얼마나 자세하게 정보를 표시할 것인가를 지정한다. (0, 1, 2)

# mode : 관찰 항목에 대해 개선이 없다고 판단하기 위한 기준을 지정한다. - 다음과 같은 하위항목(auto,min,max)을 따른다.
# 예를 들어 관찰 항목이 ‘val_loss’인 경우에는 감소되는 것이 멈출 때 종료되어야 하므로, ‘min’으로 설정됨.
    # auto : 관찰하는 이름에 따라 자동으로 지정한다.
    # min : 관찰하고 있는 항목이 감소되는 것을 멈출 때 종료한다.
    # max : 관찰하고 있는 항목이 증가되는 것을 멈출 때 종료한다.

In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100, callbacks=[early_stopping]) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 131us/sample - loss: 1.8198 - acc: 0.2518 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8019 - val_acc: 0.2522 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 3s 121us/sample - loss: 1.7983 - acc: 0.2580 - macro_f1score: 9.7680e-05 - weighted_f1score: 9.1575e-06 - val_loss: 1.8010 - val_acc: 0.2530 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 3s 118us/sample - loss: 1.7935 - acc: 0.2604 - macro_f1score: 7.4697e-05 - weighted_f1score: 9.3371e-06 - val_loss: 1.7948 - val_acc: 0.2547 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 3s 118us/sample - loss: 1.7891 - acc: 0.2629 - macro_f1score: 7.9333e-04 - weighted_f1score

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 46us/sample - loss: 1.7944 - acc: 0.2625 - macro_f1score: 0.0126 - weighted_f1score: 0.0016

Accuracy: 0.2625, Macro F1 Score: 0.0126, Weighted F1 Score: 0.0016


#### 2) Size = 48

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,1))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2304)              0         
_________________________________________________________________
d1 (Dense)                   (None, 128)               295040    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
d2 (Dense)                   (None, 512)               66048     
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
d3 (Dense)                   (None, 7)                 3591

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')


In [0]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100, callbacks=[early_stopping]) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 140us/sample - loss: 1.8201 - acc: 0.2480 - macro_f1score: 0.0013 - weighted_f1score: 3.1486e-04 - val_loss: 1.7793 - val_acc: 0.2641 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 4s 125us/sample - loss: 1.7765 - acc: 0.2741 - macro_f1score: 0.0111 - weighted_f1score: 0.0013 - val_loss: 1.7705 - val_acc: 0.2686 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 4s 128us/sample - loss: 1.7655 - acc: 0.2845 - macro_f1score: 0.0175 - weighted_f1score: 0.0020 - val_loss: 1.7382 - val_acc: 0.2990 - val_macro_f1score: 0.0238 - val_weighted_f1score: 0.0027
Epoch 4/100
28698/28698 [==============================] - 4s 128us/sample - loss: 1.7665 - acc: 0.2869 - macro_f1score: 0.0203 - weighted_f1score: 0.0022 - val_loss: 1.7318 - va

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 44us/sample - loss: 1.7400 - acc: 0.2985 - macro_f1score: 0.0060 - weighted_f1score: 6.2382e-04

Accuracy: 0.2985, Macro F1 Score: 0.0060, Weighted F1 Score: 0.0006


## 2. Simple CNN
---

In [0]:
# FER2013 데이터를 들어가기 앞서, 주의해야할 사항이 있다.
# 기본이기도 하지만, 데이터가 Label에 대해 균형이어야 좋은 performance를 낸다. 이는 CNN에서 특히 더그런다.
# 내가가진 데이터는 불균형이므로, 불균형문제를 해결하는게 옳다.
# 이 방법은 ADASYN, SMOTE, GAN, AutoEncoder등으로 해결할 수 있으며, 이에 대한 개념을 훗날 배우고 더 적용시켜보도록 한다.
# 또한, 불균형이기에 F1 score도 같이 제시함을 잊지 않는다.

###################### 2. model - CNN Simple ######################

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
#data handling
x_train = np.array(x_train).reshape([-1,48,48,1]) / 255
x_valid = np.array(x_valid).reshape([-1,48,48,1]) / 255
x_test = np.array(x_test).reshape([-1,48,48,1]) / 255

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
epochs = 100
classes = y_test.shape[1]
size=64

input_shape = (size,size,1)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 64
x_train = np.array(x_train).reshape([-1,48,48,1])

In [0]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_train_zoom.shape

(28698, 64, 64, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_valid_zoom.shape

(3589, 64, 64, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_test_zoom.shape

(3588, 64, 64, 1)

#### 1) Size 64

In [0]:
# CNN model
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer2'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model.add(tf.keras.layers.Dropout(dropout))
cnn_model.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 62, 62, 64)        640       
_________________________________________________________________
Maxpooling1_2D (MaxPooling2D (None, 31, 31, 64)        0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 29, 29, 64)        36928     
_________________________________________________________________
Maxpooling2_2D (MaxPooling2D (None, 14, 14, 64)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 12544)             0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 64)               

In [0]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')

In [0]:
cnn_model.fit(x_train_zoom,y_train, validation_data=(x_valid_zoom,y_valid) ,batch_size=batch_size,epochs=epochs,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 216s 8ms/sample - loss: 1.8178 - acc: 0.2505 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8159 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 217s 8ms/sample - loss: 1.8127 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8241 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 215s 7ms/sample - loss: 1.8128 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8167 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 215s 7ms/sample - loss: 1.8119 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score

In [0]:
_, acc, mac_f1, wei_f1 = cnn_model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 7s 2ms/sample - loss: 1.8126 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


#### 2) Size = 48

In [0]:
epochs = 100
classes = y_test.shape[1]
size=48

input_shape = (size,size,1)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 46, 46, 64)        640       
_________________________________________________________________
Maxpooling1_2D (MaxPooling2D (None, 23, 23, 64)        0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 21, 21, 64)        36928     
_________________________________________________________________
Maxpooling2_2D (MaxPooling2D (None, 10, 10, 64)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 6400)              0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 64)               

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')

In [0]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 128s 4ms/sample - loss: 1.7034 - acc: 0.3182 - macro_f1score: 0.0369 - weighted_f1score: 0.0072 - val_loss: 1.5660 - val_acc: 0.3945 - val_macro_f1score: 0.0925 - val_weighted_f1score: 0.0186
Epoch 2/100
28698/28698 [==============================] - 126s 4ms/sample - loss: 1.5243 - acc: 0.4166 - macro_f1score: 0.1410 - weighted_f1score: 0.0263 - val_loss: 1.4413 - val_acc: 0.4503 - val_macro_f1score: 0.1674 - val_weighted_f1score: 0.0318
Epoch 3/100
28698/28698 [==============================] - 127s 4ms/sample - loss: 1.4197 - acc: 0.4585 - macro_f1score: 0.2023 - weighted_f1score: 0.0367 - val_loss: 1.3830 - val_acc: 0.4742 - val_macro_f1score: 0.2174 - val_weighted_f1score: 0.0400
Epoch 4/100
28698/28698 [==============================] - 128s 4ms/sample - loss: 1.3554 - acc: 0.4866 - macro_f1score: 0.2463 - weighted_f1score: 0.0440 - val_loss: 1.3417 - val_acc: 0.4809 - val_

In [0]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 4s 1ms/sample - loss: 1.3117 - acc: 0.5524 - macro_f1score: 0.4564 - weighted_f1score: 0.0726

Accuracy: 0.5524, Macro F1 Score: 0.4564, Weighted F1 Score: 0.0726


## 3. Size = 48 , No Early Stopping
---

### 1) Epoch = 50

#### (1) MLP

In [16]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,1))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [19]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 5s 158us/sample - loss: 1.8230 - acc: 0.2448 - macro_f1score: 0.0020 - weighted_f1score: 4.6628e-04 - val_loss: 1.8039 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
28698/28698 [==============================] - 4s 145us/sample - loss: 1.8097 - acc: 0.2512 - macro_f1score: 9.4587e-04 - weighted_f1score: 2.3819e-04 - val_loss: 1.8124 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
28698/28698 [==============================] - 4s 139us/sample - loss: 1.8015 - acc: 0.2506 - macro_f1score: 0.0037 - weighted_f1score: 9.2130e-04 - val_loss: 1.8174 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/50
28698/28698 [==============================] - 4s 149us/sample - loss: 1.8103 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

In [20]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 43us/sample - loss: 1.8025 - acc: 0.2678 - macro_f1score: 0.0029 - weighted_f1score: 5.7879e-04

Accuracy: 0.2678, Macro F1 Score: 0.0029, Weighted F1 Score: 0.0006


#### (2) Simple CNN

In [0]:
# early stopping 없이 
epochs = 50
classes = y_test.shape[1]
size=48

input_shape = (size,size,1)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [24]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 122s 4ms/sample - loss: 1.7023 - acc: 0.3241 - macro_f1score: 0.0401 - weighted_f1score: 0.0076 - val_loss: 1.5833 - val_acc: 0.3931 - val_macro_f1score: 0.1149 - val_weighted_f1score: 0.0213
Epoch 2/50
28698/28698 [==============================] - 122s 4ms/sample - loss: 1.5300 - acc: 0.4142 - macro_f1score: 0.1363 - weighted_f1score: 0.0255 - val_loss: 1.4600 - val_acc: 0.4542 - val_macro_f1score: 0.1529 - val_weighted_f1score: 0.0282
Epoch 3/50
28698/28698 [==============================] - 123s 4ms/sample - loss: 1.4329 - acc: 0.4515 - macro_f1score: 0.1872 - weighted_f1score: 0.0341 - val_loss: 1.3901 - val_acc: 0.4687 - val_macro_f1score: 0.1880 - val_weighted_f1score: 0.0346
Epoch 4/50
28698/28698 [==============================] - 123s 4ms/sample - loss: 1.3665 - acc: 0.4807 - macro_f1score: 0.2310 - weighted_f1score: 0.0412 - val_loss: 1.3354 - val_acc: 0.4932 - val_macr

In [25]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 4s 1ms/sample - loss: 1.5850 - acc: 0.5438 - macro_f1score: 0.4679 - weighted_f1score: 0.0754

Accuracy: 0.5438, Macro F1 Score: 0.4679, Weighted F1 Score: 0.0754


### 2) Epoch = 100

#### 1) MLP

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,1))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [29]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 138us/sample - loss: 1.8212 - acc: 0.2429 - macro_f1score: 0.0033 - weighted_f1score: 8.2958e-04 - val_loss: 1.7987 - val_acc: 0.2446 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 4s 128us/sample - loss: 1.7874 - acc: 0.2497 - macro_f1score: 0.0143 - weighted_f1score: 0.0036 - val_loss: 1.8148 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 4s 131us/sample - loss: 1.8037 - acc: 0.2511 - macro_f1score: 0.0014 - weighted_f1score: 3.3154e-04 - val_loss: 1.8087 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 4s 132us/sample - loss: 1.7972 - acc: 0.2513 - macro_f1score: 0.0062 - weighted_f1score: 0.0015 - val_loss:

In [30]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 74us/sample - loss: 1.7322 - acc: 0.2768 - macro_f1score: 0.0322 - weighted_f1score: 0.0079

Accuracy: 0.2768, Macro F1 Score: 0.0322, Weighted F1 Score: 0.0079


#### (2) Simple CNN

In [0]:
# early stopping 없이 
epochs = 100
classes = y_test.shape[1]
size=48

input_shape = (size,size,1)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [34]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 124s 4ms/sample - loss: 1.7014 - acc: 0.3214 - macro_f1score: 0.0385 - weighted_f1score: 0.0074 - val_loss: 1.5827 - val_acc: 0.4004 - val_macro_f1score: 0.0968 - val_weighted_f1score: 0.0186
Epoch 2/100
28698/28698 [==============================] - 125s 4ms/sample - loss: 1.5178 - acc: 0.4171 - macro_f1score: 0.1448 - weighted_f1score: 0.0268 - val_loss: 1.4537 - val_acc: 0.4394 - val_macro_f1score: 0.1727 - val_weighted_f1score: 0.0325
Epoch 3/100
28698/28698 [==============================] - 124s 4ms/sample - loss: 1.4197 - acc: 0.4616 - macro_f1score: 0.2018 - weighted_f1score: 0.0366 - val_loss: 1.3764 - val_acc: 0.4678 - val_macro_f1score: 0.2080 - val_weighted_f1score: 0.0370
Epoch 4/100
28698/28698 [==============================] - 124s 4ms/sample - loss: 1.3608 - acc: 0.4808 - macro_f1score: 0.2391 - weighted_f1score: 0.0428 - val_loss: 1.3435 - val_acc: 0.4876 - val_

In [35]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 4s 1ms/sample - loss: 2.0535 - acc: 0.5477 - macro_f1score: 0.4923 - weighted_f1score: 0.0763

Accuracy: 0.5477, Macro F1 Score: 0.4923, Weighted F1 Score: 0.0763
